In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import random

from sklearn import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (accuracy_score,
                             precision_recall_fscore_support,
                             confusion_matrix,
                             ConfusionMatrixDisplay)

from keras.utils import plot_model
from keras import Model
from keras.layers import (Conv1D,
                          Dense,
                          Dropout,
                          Embedding,
                          Flatten,
                          GlobalMaxPooling1D,
                          Input,
                          LSTM,
                          TextVectorization)

df = pd.read_csv("survey2.csv")
df = df.drop('How do you usually relieve your stress', axis=1)
df = df.drop('Timestamp', axis=1)
df = df.drop('Name', axis=1)
df = df.drop('Meditation/gym/yoga or any other physical activity', axis=1)

df.rename(columns = {'Please select the option that best describes your current or past treatment/counseling status regarding mental health or emotional well-being:':'Past_Treatment'}, inplace = True)
df.rename(columns = {'Do you have any family history of mental illnesses? Please select all that apply.':'Family_History'}, inplace = True)
df.rename(columns = {'How often do you hang out with friends?':'Friends'}, inplace = True)
df.rename(columns = {'Do share the secrets and techniques you use to cope with stress. Please select all that apply.':'Techniques'}, inplace = True)
df.rename(columns = {'Have you ever taken to alcohol or drugs to distract yourself?':'Alcohol_or_drugs'}, inplace = True)
df.rename(columns = {'Sadness is a natural part of life. How frequently do you experience feelings of sadness? ':'Sadness'}, inplace = True)
df.rename(columns = {'How frequently do you experience feelings of irritability or anger?':'Anger'}, inplace = True)
df.rename(columns = {"We're curious to know:":"We're_curious_to_know"}, inplace = True)
df.rename(columns = {'Does your current organization provide any mental health support or resources?':'Support'}, inplace = True)
df.rename(columns = {"Sometimes we engage in activities that we know aren't the best for our mental health, but we still find ourselves doing them. Is there any activity or habit you frequently partake in, even though you're aware it may have a negative impact on your mental well-being?":'Negative_activities'}, inplace = True)
df.rename(columns = {'Have you ever faced any significant traumatic events? This could include events such as accidents, natural disasters, violence, or other distressing experiences.':'Trauma'}, inplace = True)
df.rename(columns = {"It's totally normal to have triggers. What's something that triggers your mindset i.e something that makes you uncomfortable or reminds you of a difficult time?":'Triggers'}, inplace = True)
df.rename(columns = {'Finally, do you think you have had any mental illness before?':'Mental_illness'}, inplace = True)

df['Negative_activities']=df['Negative_activities'].str.lower()
df['Triggers']=df['Triggers'].str.lower()
df['Trauma']=df['Trauma'].str.lower()
df['Techniques']=df['Techniques'].str.lower()

df['Negative_activities'] = df['Negative_activities'].str.replace(r'[^\w\s]+', '')
df['Triggers'] = df['Triggers'].str.replace(r'[^\w\s]+', '')
df['Trauma'] = df['Trauma'].str.replace(r'[^\w\s]+', '')

display(df)

<ipython-input-104-0eca0a88828f>:54: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Negative_activities'] = df['Negative_activities'].str.replace(r'[^\w\s]+', '')
<ipython-input-104-0eca0a88828f>:55: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Triggers'] = df['Triggers'].str.replace(r'[^\w\s]+', '')
<ipython-input-104-0eca0a88828f>:56: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Trauma'] = df['Trauma'].str.replace(r'[^\w\s]+', '')


,Age,Gender,Employment,Past_Treatment,Family_History,Friends,We're_curious_to_know,Techniques,Alcohol_or_drugs,Sadness,Anger,Support,Negative_activities,Trauma,Triggers,Mental_illness,label
0,20,Female,Student,I have not undergone any treatment or counseling.,No,4,Love is in the air!,"a visit to the gym, spending time with friends...",Yes,3,2,No,social media like scrolling on instagram,yes,the siren of an ambulance,Not any as such,1
1,21,Male,Student,I have not undergone any treatment or counseling.,I do not have any family history of medical di...,3,Love is in the air!,"spending time with friends, going out with the...",No,2,3,No,no,no,when someone speaks wrongly about my character...,Not any as such,1
2,20,Male,Student,I have not undergone any treatment or counseling.,None,4,Cupid's arrow missed its mark :(,"a visit to the gym, listen to some music",NaN,3,4,Yes,NaN,NaN,NaN,Not any as such,0
3,21,Female,Student,I have undergone treatment or counseling in th...,Anxiety,4,Its complicated.,"spending time with friends, going out with the...",No,3,4,Yes,no,yes,fights and arguments,Not any as such,1
4,20,Male,Student,I have not undergone any treatment or counseling.,None,3,Cupid's arrow missed its mark :(,"spending time with friends, going out with the...","No, I consume in very limited quantity only on...",2,2,Yes,i dont think so,none,not that i know of,Not any as such,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,20,Female,Student,I am currently undergoing treatment or counsel...,"Anxiety, Depression",3,None of the above,"listen to some music, journaling, positive sel...",Yes,2,4,Yes,procrastination,NaN,NaN,Not any as such,0
217,20,Female,Student,I have not undergone any treatment or counseling.,not any as such,5,None of the above,"spending time with friends, going out with the...",Nah,4,5,No,no,no,my school days,Depression,1
218,20,Female,Unemployed,I have not undergone any treatment or counseling.,not any as such,2,None of the above,"spending time with friends, going out with the...",No,3,4,Yes,no,no,when someone makes me feel left out or belittl...,Not any as such,1
219,19,Male,Student,I am currently undergoing treatment or counsel...,Depression,3,Love is in the air!,"meditation and yoga, binge on shows and movies...",No,4,3,No,NaN,NaN,NaN,Bipolar disorder,0


In [ ]:
df['Negative_activities'] = df['Negative_activities'].fillna("no")
df['Alcohol_or_drugs'] = df['Alcohol_or_drugs'].fillna("no")
df['Trauma'] = df['Trauma'].fillna("no")
df['Triggers'] = df['Triggers'].fillna("no")
df['Family_History'] = df['Family_History'].fillna("no")
display(df)
my_df = df

,Age,Gender,Employment,Past_Treatment,Family_History,Friends,We're_curious_to_know,Techniques,Alcohol_or_drugs,Sadness,Anger,Support,Negative_activities,Trauma,Triggers,Mental_illness,label
0,20,Female,Student,I have not undergone any treatment or counseling.,No,4,Love is in the air!,"a visit to the gym, spending time with friends...",Yes,3,2,No,social media like scrolling on instagram,yes,the siren of an ambulance,Not any as such,1
1,21,Male,Student,I have not undergone any treatment or counseling.,I do not have any family history of medical di...,3,Love is in the air!,"spending time with friends, going out with the...",No,2,3,No,no,no,when someone speaks wrongly about my character...,Not any as such,1
2,20,Male,Student,I have not undergone any treatment or counseling.,None,4,Cupid's arrow missed its mark :(,"a visit to the gym, listen to some music",no,3,4,Yes,no,no,no,Not any as such,0
3,21,Female,Student,I have undergone treatment or counseling in th...,Anxiety,4,Its complicated.,"spending time with friends, going out with the...",No,3,4,Yes,no,yes,fights and arguments,Not any as such,1
4,20,Male,Student,I have not undergone any treatment or counseling.,None,3,Cupid's arrow missed its mark :(,"spending time with friends, going out with the...","No, I consume in very limited quantity only on...",2,2,Yes,i dont think so,none,not that i know of,Not any as such,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,20,Female,Student,I am currently undergoing treatment or counsel...,"Anxiety, Depression",3,None of the above,"listen to some music, journaling, positive sel...",Yes,2,4,Yes,procrastination,no,no,Not any as such,0
217,20,Female,Student,I have not undergone any treatment or counseling.,not any as such,5,None of the above,"spending time with friends, going out with the...",Nah,4,5,No,no,no,my school days,Depression,1
218,20,Female,Unemployed,I have not undergone any treatment or counseling.,not any as such,2,None of the above,"spending time with friends, going out with the...",No,3,4,Yes,no,no,when someone makes me feel left out or belittl...,Not any as such,1
219,19,Male,Student,I am currently undergoing treatment or counsel...,Depression,3,Love is in the air!,"meditation and yoga, binge on shows and movies...",No,4,3,No,no,no,no,Bipolar disorder,0


In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Gender']= label_encoder.fit_transform(df['Gender'])
df['Employment']= label_encoder.fit_transform(df['Employment'])
df['Past_Treatment']= label_encoder.fit_transform(df['Past_Treatment'])
df['Support']= label_encoder.fit_transform(df['Support'])
df["We're_curious_to_know"]= label_encoder.fit_transform(df["We're_curious_to_know"])
df["Techniques"]= label_encoder.fit_transform(df["Techniques"])

In [ ]:
df['Mental_illness'] = df['Mental_illness'].replace(['Big wood sickness '], 'Not any as such')
df['Mental_illness'] = df['Mental_illness'].replace(['Not any as such, During exams. Stomach mai gud gud '], 'Not any as such')
df['Mental_illness'] = df['Mental_illness'].replace(['Anger, overthinking '], 'Not any as such')
df['Mental_illness'] = df['Mental_illness'].replace(['PTSD', 'Not any as such, '], 'PTSD')
df['Mental_illness'] = df['Mental_illness'].replace(['Anxiety, Not any as such'], 'Anxiety')
#df['Mental illness']= label_encoder.fit_transform(df['Mental illness'])

alc = df['Alcohol_or_drugs'].tolist()

for i in range(len(alc)):
  if (alc[i] == 'No' or alc[i] == 'no' or alc[i] == 'Nope' or alc[i] == 'No. Driving is all I need. '
  or alc[i] == 'Nah' or alc[i] == 'Never to distract' or alc[i] == 'Never' or alc[i] == 'No Never ' or alc[i] == 'NO'
  or alc[i] == 'nope' or alc[i] == 'Nahhh' or alc[i] == 'Nay' or alc[i] == 'Not even once ' or alc[i] == 'Nope, not interested'
  or alc[i] == 'haha no :)' or alc[i] == "I'm 14? No?"):
    alc[i] = 0

  else:
    alc[i] = 1

df['Alcohol_or_drugs'] = alc

display(df)


,Age,Gender,Employment,Past_Treatment,Family_History,Friends,We're_curious_to_know,Techniques,Alcohol_or_drugs,Sadness,Anger,Support,Negative_activities,Trauma,Triggers,Mental_illness,label
0,20,0,3,1,No,4,3,2,1,3,2,0,social media like scrolling on instagram,yes,the siren of an ambulance,Not any as such,1
1,21,1,3,1,I do not have any family history of medical di...,3,3,76,0,2,3,0,no,no,when someone speaks wrongly about my character...,Not any as such,1
2,20,1,3,1,None,4,0,0,0,3,4,1,no,no,no,Not any as such,0
3,21,0,3,2,Anxiety,4,2,81,0,3,4,1,no,yes,fights and arguments,Not any as such,1
4,20,1,3,1,None,3,0,79,1,2,2,1,i dont think so,none,not that i know of,Not any as such,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,20,0,3,0,"Anxiety, Depression",3,4,35,1,2,4,1,procrastination,no,no,Not any as such,0
217,20,0,3,1,not any as such,5,4,85,0,4,5,0,no,no,my school days,Depression,1
218,20,0,4,1,not any as such,2,4,83,0,3,4,1,no,no,when someone makes me feel left out or belittl...,Not any as such,1
219,19,1,3,0,Depression,3,3,41,0,4,3,0,no,no,no,Bipolar disorder,0


In [ ]:
illness = df['Mental_illness'].tolist()
#print(illness)
for i in range(len(illness)):
  if illness[i] == 'Not any as such':
    illness[i] = 0

  elif (illness[i] == 'Anxiety' or illness[i] == 'Depression' or illness[i] == 'PTSD' or
    illness[i] == 'Anxiety, Depression' or illness[i] == 'Anxiety, PTSD' or illness[i] == 'Depression, PTSD'
    or illness[i] == 'Anxiety, Depression, PTSD' or illness[i] == 'Anxiety, High Stress' or
    illness[i] == 'Anxiety, Depression, PTSD, Schizophrenia ' or illness[i] == 'Bipolar disorder '):
      illness[i] = 1

df['Mental_illness'] = illness

trauma = df['Trauma'].tolist()


for i in range(len(trauma)):
  if (trauma[i] == 'No' or trauma[i] == 'no' or trauma[i] == 'none' or trauma[i] == 'not in the recent past' or trauma[i] == 'not really ' or
      trauma[i] == 'nothing as such' or trauma[i] == 'not yet' or trauma[i] == 'nope' or trauma[i] == 'nop' or trauma[i] == 'luckily nothing major' or
      trauma[i] == 'nay' or trauma[i] == 'not really any ' or trauma[i] == 'noo'):
        trauma[i] = 0

  else:
    trauma[i] = 1


df['Trauma'] = trauma



In [ ]:
import re

answer1 = df['Triggers'].tolist()
answer2 = df['Negative_activities'].tolist()
answer3 = df['Family_History'].tolist()

no_words = ['no', 'nope', 'not', 'negative', 'none', 'neither', 'never', 'nowhere', 'nobody', 'nothing', 'dont']
trigger_values = [0 if re.search(r'\b(?:{})\b'.format('|'.join(no_words)), answer.lower()) else 1 for answer in answer1]
neg_act_values = [0 if re.search(r'\b(?:{})\b'.format('|'.join(no_words)), answer.lower()) else 1 for answer in answer2]
fam = [0 if re.search(r'\b(?:{})\b'.format('|'.join(no_words)), answer.lower()) else 1 for answer in answer3]

df['Triggers'] = trigger_values
df['Negative_activities'] = neg_act_values
df['Family_History'] = fam
display(df)


,Age,Gender,Employment,Past_Treatment,Family_History,Friends,We're_curious_to_know,Techniques,Alcohol_or_drugs,Sadness,Anger,Support,Negative_activities,Trauma,Triggers,Mental_illness,label
0,20,0,3,1,0,4,3,2,1,3,2,0,1,1,1,0,1
1,21,1,3,1,0,3,3,76,0,2,3,0,0,0,1,0,1
2,20,1,3,1,0,4,0,0,0,3,4,1,0,0,0,0,0
3,21,0,3,2,1,4,2,81,0,3,4,1,0,1,1,0,1
4,20,1,3,1,0,3,0,79,1,2,2,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,20,0,3,0,1,3,4,35,1,2,4,1,1,0,0,0,0
217,20,0,3,1,0,5,4,85,0,4,5,0,0,0,1,1,1
218,20,0,4,1,0,2,4,83,0,3,4,1,0,0,1,0,1
219,19,1,3,0,1,3,3,41,0,4,3,0,0,0,0,1,0


In [ ]:
# Calculate the minimum and maximum values of the column
min_val = df['Techniques'].min()
max_val = df['Techniques'].max()

# Apply Min-Max Scaling formula: (x - min) / (max - min)
df['Techniques'] = (df['Techniques'] - min_val) / (max_val - min_val)

display(df)



,Age,Gender,Employment,Past_Treatment,Family_History,Friends,We're_curious_to_know,Techniques,Alcohol_or_drugs,Sadness,Anger,Support,Negative_activities,Trauma,Triggers,Mental_illness,label
0,20,0,3,1,0,4,3,0.013158,1,3,2,0,1,1,1,0,1
1,21,1,3,1,0,3,3,0.500000,0,2,3,0,0,0,1,0,1
2,20,1,3,1,0,4,0,0.000000,0,3,4,1,0,0,0,0,0
3,21,0,3,2,1,4,2,0.532895,0,3,4,1,0,1,1,0,1
4,20,1,3,1,0,3,0,0.519737,1,2,2,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,20,0,3,0,1,3,4,0.230263,1,2,4,1,1,0,0,0,0
217,20,0,3,1,0,5,4,0.559211,0,4,5,0,0,0,1,1,1
218,20,0,4,1,0,2,4,0.546053,0,3,4,1,0,0,1,0,1
219,19,1,3,0,1,3,3,0.269737,0,4,3,0,0,0,0,1,0


In [ ]:

# basic logistic regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

data = df

X = data.drop("Mental_illness", axis=1)
y = data["Mental_illness"]

#X = data.iloc[:,:-1].values
#y = data.iloc[:,-1].values

print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_scaled)
print(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))



     Age  Gender  Employment  Past_Treatment  Family_History  Friends  \
0     20       0           3               1               0        4   
1     21       1           3               1               0        3   
2     20       1           3               1               0        4   
3     21       0           3               2               1        4   
4     20       1           3               1               0        3   
..   ...     ...         ...             ...             ...      ...   
216   20       0           3               0               1        3   
217   20       0           3               1               0        5   
218   20       0           4               1               0        2   
219   19       1           3               0               1        3   
220   20       0           3               1               0        5   

     We're_curious_to_know  Techniques  Alcohol_or_drugs  Sadness  Anger  \
0                        3    0.013158         

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
'''
# this is svm thingy,while testing it try to change kernel to linear, polynomial, radial and keep testing.regularisation is done alreday
# importing scikit learn with make_blobs
from sklearn.datasets import make_blobs

# creating datasets X containing n_samples
# Y containing two classes
X, Y = make_blobs(n_samples=500, centers=2,
				random_state=0, cluster_std=0.40)

# plotting scatters
plt.scatter(X[:, 0], X[:, 1], c=Y, s=50, cmap='spring');
plt.show()

# creating linspace between -1 to 3.5
xfit = np.linspace(-1, 3.5)

# plotting scatter
plt.scatter(X[:, 0], X[:, 1], c=Y, s=50, cmap='spring')

# plot a line between the different sets of data
for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
	yfit = m * xfit + b
	plt.plot(xfit, yfit, '-k')
	plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
	color='#AAAAAA', alpha=0.4)

plt.xlim(-1, 3.5);
plt.show()
'''



"\n# this is svm thingy,while testing it try to change kernel to linear, polynomial, radial and keep testing.regularisation is done alreday\n# importing scikit learn with make_blobs\nfrom sklearn.datasets import make_blobs\n\n# creating datasets X containing n_samples\n# Y containing two classes\nX, Y = make_blobs(n_samples=500, centers=2,\n\t\t\t\trandom_state=0, cluster_std=0.40)\n\n# plotting scatters\nplt.scatter(X[:, 0], X[:, 1], c=Y, s=50, cmap='spring');\nplt.show()\n\n# creating linspace between -1 to 3.5\nxfit = np.linspace(-1, 3.5)\n\n# plotting scatter\nplt.scatter(X[:, 0], X[:, 1], c=Y, s=50, cmap='spring')\n\n# plot a line between the different sets of data\nfor m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:\n\tyfit = m * xfit + b\n\tplt.plot(xfit, yfit, '-k')\n\tplt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',\n\tcolor='#AAAAAA', alpha=0.4)\n\nplt.xlim(-1, 3.5);\nplt.show()\n"

In [ ]:

# reading csv file and extracting class column to y.
x = df
a = np.array(x)
y = a[:,-1] # classes having 0 and 1

# extracting two features
x = np.column_stack((x.Age, x.Gender, x.Employment, x['Past_Treatment'], x.Family_History, x.Friends, x["We're_curious_to_know"], x.Techniques, x['Alcohol_or_drugs'], x.Sadness, x.Anger, x.Support, x['Negative_activities'], x.Trauma, x.Triggers))

x.shape

print (x),(y)

# import support vector classifier
# "Support Vector Classifier"

from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1.0, gamma='scale')

# fitting x samples and y classes
clf.fit(x, y)
pred1 = clf.predict([[20, 1, 3, 0, 0, 4, 2, 0.5, 1, 2, 2, 0, 1, 1, 1]])
print("Prediction 1:", pred1)
if (pred1 == 1):
  print("Person is likely to have a mental illness (SVC)")

else:
  print("Person is not likely to have a mental illness (SVC)")

#clf.predict([[85, 550]])

#Age gender employment past treatment family history friends curious techniques alc sadness anger support negact trauma triggers

[[20.  0.  3. ...  1.  1.  1.]
 [21.  1.  3. ...  0.  0.  1.]
 [20.  1.  3. ...  0.  0.  0.]
 ...
 [20.  0.  4. ...  0.  0.  1.]
 [19.  1.  3. ...  0.  0.  0.]
 [20.  0.  3. ...  0.  0.  1.]]
Prediction 1: [1.]
Person is likely to have a mental illness (SVC)


In [ ]:
# this is random forest method with multiiple decision trees,test it out and remove that last predict mental health part ig its pretty stupid to keep that

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


data = df
data = data.drop('label', axis=1)

X = data.drop('Mental_illness', axis=1)
y = data['Mental_illness']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_adaboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0]
}


rf_clf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(rf_clf, param_grid_rf, cv=5)
grid_search_rf.fit(X_train, y_train)


best_rf_clf = grid_search_rf.best_estimator_


best_rf_clf.fit(X_train, y_train)


y_pred_rf = best_rf_clf.predict(X_test)


accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy of the Random Forest model:", accuracy_rf)


adaboost_clf = AdaBoostClassifier(random_state=42)
grid_search_adaboost = GridSearchCV(adaboost_clf, param_grid_adaboost, cv=5)
grid_search_adaboost.fit(X_train, y_train)


best_adaboost_clf = grid_search_adaboost.best_estimator_


best_adaboost_clf.fit(X_train, y_train)


sample_data = pd.DataFrame([
    [20, 1, 3, 0, 1, 2, 2, 0.5, 1, 3, 3, 0, 1, 1, 1],
    [26, 0, 2, 1, 0, 3, 3, 0, 1, 3, 1, 0, 0, 0, 1],

], columns=['Age','Gender','Employment','Past_Treatment','Family_History','Friends',"We're_curious_to_know",'Techniques', 'Alcohol_or_drugs', 'Sadness', 'Anger', 'Support','Negative_activities','Trauma','Triggers'])

adaboost_predictions = best_adaboost_clf.predict(sample_data)
for i, prediction in enumerate(adaboost_predictions):
    if prediction == 1:
        print(f"Sample {i+1}: The person is likely undergoing a mental health disorder (AdaBoost).")
    else:
        print(f"Sample {i+1}: The person is not likely to undergo a mental health disorder (AdaBoost).")

#Age gender employment past_treatment family_history friends curious techniques alc sadness anger support negact trauma triggers mental illness label

Accuracy of the Random Forest model: 0.6567164179104478
Sample 1: The person is likely undergoing a mental health disorder (AdaBoost).
Sample 2: The person is not likely to undergo a mental health disorder (AdaBoost).
